In [68]:
import pandas as pd
import numpy as np
import pyreadstat

In [69]:
gdps = pd.read_csv("/home/vpaz/Downloads/total_gdp_us_inflation_adjusted.csv")
gdps = gdps[["country", *[str(year) for year in range(1995, 2019)]]].set_index("country")

def to_number(x):
    if isinstance(x, (float, int)):
        return x
    if "B" in x:
        return float(x[:-1]) * 1_000_000_000
    elif "M" in x:
        return float(x[:-1]) * 1_000_000
    else:
        try:
            return float(x)
        except ValueError:
            return np.NaN
    
for year in range(1995, 2019):
    gdps[str(year)] = gdps[str(year)].map(to_number)

# desicion: usar el interpolate para completar datos faltantes
gdps = gdps.interpolate()

In [70]:
dtafile = "/home/vpaz/Desktop/location.dta"
locations_gdp_df, meta = pyreadstat.read_dta(dtafile)
locations_gdp_df = locations_gdp_df.merge(gdps, right_on="country", left_on="location_name_short_en")

In [71]:
df = pd.read_csv("data/exports.csv", index_col=0)
df = df.drop(columns=["partner_id", "year", "import_value", "sitc_eci", "sitc_coi", "partner_code", "sitc_product_code"], errors="ignore") \
    .groupby(["location_id", "location_code", "product_id"], as_index=False).sum()

products_df = pd.read_csv("data/datos_productos.csv")


/home/vpaz/envs/sed/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [72]:
products_df = products_df[pd.to_numeric(products_df.producto_code, errors='coerce').notnull()]

products_df.producto_code = pd.to_numeric(products_df.producto_code)
merged_products_df = products_df.merge(df, left_on="producto_code", right_on="product_id")
products = frozenset((x.label, x.producto_code, x.initialVolume, x.b, x.t0) for _, x in merged_products_df[["label", "producto_code", "initialVolume", "b", "t0"]].iterrows())
pd.DataFrame(data=products, columns=["label", "producto_code", "initialVolume", "b", "t0"]).to_csv("products.csv")

In [73]:
merged_products_df.drop(columns=["producto_code", "initialVolume", "b", "t0"])
countries_df = merged_products_df.drop(columns=["producto_code", "initialVolume", "b", "t0"]).merge(locations_gdp_df[["location_id", "location_code", "1995"]]) # implicit on location_id
product_codes = sorted(list(frozenset(countries_df.label)))
countries = frozenset((x.location_id, x.location_code, x["1995"]) for _, x in countries_df[["location_id", "location_code", "1995"]].iterrows())
countries_df = countries_df[["location_id", "label", "export_value"]].groupby(["location_id", "label"]).sum()
# pd.DataFrame(
#     data=,
#     columns=["location_id", "location_code", "1995"],
# ).to_csv("locations_gdp.csv")

In [74]:
countries_info = []
for x in countries:
    location_id = x[0]
    initial_gdp = x[2]
    country_info = list(x)
    country = countries_df.loc[location_id]
    country_exports = [country.loc[p]["export_value"] if p in country.index else 0 for p in product_codes]

    sum_country_exports = sum(country_exports)
    if sum_country_exports > 0:
        countries_info.append(country_info + [initial_gdp / sum_country_exports] + country_exports)

pd.DataFrame(data=countries_info, columns=["location_id", "location_code", "1995", "gdp_over_exports", *product_codes]).to_csv("countries.csv")
